In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import warnings
warnings.filterwarnings(action='ignore')
from catboost import *
import sklearn
import math
from sklearn.preprocessing import LabelEncoder
import math


def log(train,test):
    col_list = train.columns
    for col in col_list :
        if '제곱' in col:
            train[col+'log'] = np.log1p(train[col])
            test[col+'log'] = np.log1p(test[col])
    return train,test

def del_columns(train, test):
    col_list = train.columns
    nan_list = []
    nan_cnt = []
    nan_col = []
    full_list = []
    for col in col_list:
        if train[col].isnull().sum() == 0 :
            full_list.append(col)
            continue
        nan_list.append([col, train[col].isnull().sum()])
        nan_cnt.append(train[col].isnull().sum())
        nan_col.append(col)

    '''모든값이 결측값이면 제거'''
    del_col = []
    for col in nan_list :
        if col[1] == len(train) :
            del_col.append(col[0])
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)

    '''값이 1개 존재하면 제거'''
    del_col = []
    col_list = train.describe().columns
    for col in col_list :
        if col == 'Y_Class':
            continue
        if col == 'Y_Quality':
            continue
        if col == 'LINE':
            continue
        if col == 'PRODUCT_CODE':
            continue
        if train[col].nunique()==1 :
            del_col.append(col)
    train = train.drop(columns=del_col)
    test = test.drop(columns=del_col)
    
    return train,test
   
def make_train_test_dataset(train,test):
    train_x = train.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE','Y_Class','Y_Quality'])
    test_x = test.drop(columns=['PRODUCT_ID','TIMESTAMP','PRODUCT_CODE'])
    train_y = train['Y_Quality']
    train_w = train[['Y_Class']]
    return train_x, test_x, train_y, train_w

def fillna(train,test,value):
    train = train.fillna(value)
    test = test.fillna(value)
    return train,test

def labelencoder(train,test,col_list):
    qual_col = col_list
    for i in qual_col:
        le = LabelEncoder()
        le = le.fit(train[i])
        train[i] = le.transform(train[i])

        for label in np.unique(test[i]): 
            if label not in le.classes_: 
                le.classes_ = np.append(le.classes_, label)
        test[i] = le.transform(test[i]) 
    return train,test

def multi_col(train,test) : 
    col_list = train.columns
    for col in col_list :
        if col == 'LINE':
            continue
        train[col+'제곱'] = (train[col]**2)//100
        test[col+'제곱'] = (test[col]**2)//100
    return train,test

# (1) y_q 값을 로그변환

In [75]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# y quality scaling
train['Y_Quality'] = train['Y_Quality'].map(lambda x : np.log(x))

train, test = del_columns(train,test)

trainA_31 = train[train['PRODUCT_CODE']=='A_31']
train_T_31 = train[train['PRODUCT_CODE']=='T_31']
train_O_31 = train[train['PRODUCT_CODE']=='O_31']

testA_31 = test[test['PRODUCT_CODE']=='A_31']
test_T_31 = test[test['PRODUCT_CODE']=='T_31']
test_O_31 = test[test['PRODUCT_CODE']=='O_31']

trainA_31, testA_31 = del_columns(trainA_31,testA_31)
train_T_31, test_T_31 = del_columns(train_T_31,test_T_31)
train_O_31, test_O_31 = del_columns(train_O_31,test_O_31)

trainA_31_x,testA_31_x, trainA_31_y, trainA_31_w = make_train_test_dataset(trainA_31,testA_31)
train_T_31_x,test_T_31_x, train_T_31_y, train_T_31_w = make_train_test_dataset(train_T_31,test_T_31)
train_O_31_x,test_O_31_x, train_O_31_y, train_O_31_w = make_train_test_dataset(train_O_31,test_O_31)

trainA_31_x,testA_31_x = labelencoder(trainA_31_x,testA_31_x,['LINE'])
train_T_31_x,test_T_31_x = labelencoder(train_T_31_x,test_T_31_x,['LINE'])
train_O_31_x,test_O_31_x = labelencoder(train_O_31_x,test_O_31_x,['LINE'])

trainA_31_x,testA_31_x = fillna(trainA_31_x,testA_31_x,-1)
train_T_31_x,test_T_31_x = fillna(train_T_31_x,test_T_31_x,-1)
train_O_31_x,test_O_31_x = fillna(train_O_31_x,test_O_31_x,-1)

'''
# 강한 추세성보이는 칼럼 중 일부 제거
drop_a = ['X_1167','X_1168','X_1169','X_1170','X_1171','X_1274','X_1275','X_1276','X_1277','X_1278']
drop_t = ['X_529','X_530','X_531','X_532','X_533','X_600','X_601','X_602','X_603','X_604']

trainA_31_x = trainA_31_x.drop(columns=drop_a)
testA_31_x = testA_31_x.drop(columns=drop_a)
train_T_31_x = train_T_31_x.drop(columns=drop_t)
test_T_31_x = test_T_31_x.drop(columns=drop_t)


trainA_31_x,testA_31_x = multi_col(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = multi_col(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = multi_col(train_O_31_x,test_O_31_x)

trainA_31_x,testA_31_x = log(trainA_31_x,testA_31_x)
train_T_31_x,test_T_31_x = log(train_T_31_x,test_T_31_x)
train_O_31_x,test_O_31_x = log(train_O_31_x,test_O_31_x)
'''

print(trainA_31_x.shape,testA_31_x.shape,train_T_31_x.shape,test_T_31_x.shape)

(249, 1866) (67, 1866) (343, 550) (239, 550)


In [76]:
print(train[train['Y_Class']==0]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].min(),'\n',
      train[train['Y_Class']==1]['Y_Quality'].max(),'\n',
      train[train['Y_Class']==2]['Y_Quality'].min(),'\n')

-0.6442300396904909 
 -0.6441937649548493 
 -0.6257823004654807 
 -0.6255805101477584 



In [77]:
model1 = CatBoostRegressor(random_state=1234,verbose=500,iterations=1500,learning_rate=0.033)

In [78]:
model1.fit(trainA_31_x, trainA_31_y)
pred_a = model1.predict(testA_31_x)

0:	learn: 0.0183320	total: 24.5ms	remaining: 36.7s
500:	learn: 0.0012455	total: 7.32s	remaining: 14.6s
1000:	learn: 0.0001219	total: 14.5s	remaining: 7.24s
1499:	learn: 0.0000129	total: 22.6s	remaining: 0us


In [79]:
model1.fit(train_T_31_x, train_T_31_y)
pred_t = model1.predict(test_T_31_x)

0:	learn: 0.0089112	total: 7.55ms	remaining: 11.3s
500:	learn: 0.0009494	total: 3.14s	remaining: 6.25s
1000:	learn: 0.0001266	total: 6.02s	remaining: 3s
1499:	learn: 0.0000173	total: 8.88s	remaining: 0us


In [80]:
model1.fit(train_O_31_x, train_O_31_y)
pred_o = model1.predict(test_O_31_x)

0:	learn: 0.0059697	total: 1.87ms	remaining: 2.8s
500:	learn: 0.0000906	total: 587ms	remaining: 1.17s
1000:	learn: 0.0000014	total: 1.15s	remaining: 575ms
1499:	learn: 0.0000000	total: 1.72s	remaining: 0us


In [81]:
testA_31['Y_quanlity'] = pred_a
test_T_31['Y_quanlity'] = pred_t
test_O_31['Y_quanlity'] = pred_o

testA_31['Y_Class'] = 1
test_T_31['Y_Class'] = 1
test_O_31['Y_Class'] = 1

testA_31.loc[(testA_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
testA_31.loc[(testA_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2
test_T_31.loc[(test_T_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_T_31.loc[(test_T_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2
test_O_31.loc[(test_O_31['Y_quanlity']<-0.64421190232267), 'Y_Class'] = 0
test_O_31.loc[(test_O_31['Y_quanlity']>-0.6256814053066195), 'Y_Class'] = 2

submita = pd.read_csv('sample_submission.csv')
submitt = pd.read_csv('sample_submission.csv')
submito = pd.read_csv('sample_submission.csv')

submita = pd.merge(submita[['PRODUCT_ID']],testA_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submitt = pd.merge(submitt[['PRODUCT_ID']],test_T_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')
submito = pd.merge(submito[['PRODUCT_ID']],test_O_31[['PRODUCT_ID','Y_Class']],on='PRODUCT_ID')

pd.concat([submita,submitt,submito]).sort_values(by='PRODUCT_ID').to_csv('scale1.csv',index=False)

In [70]:
df = pd.read_csv('scale1.csv')

In [71]:
df['Y_Class'].value_counts()

1    272
0     30
2      8
Name: Y_Class, dtype: int64

In [31]:
df['Y_Class'].value_counts() # 1234, 1500, 0.33

1    269
0     37
2      4
Name: Y_Class, dtype: int64

In [47]:
df[df['Y_Class']==2] # 1234, 5500, 0.02

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
160,TEST_160,2
161,TEST_161,2
177,TEST_177,2
183,TEST_183,2


In [62]:
testA_31['Y_Class'].values

array([0, 1, 1, 1, 0, 0, 2, 1, 1, 2, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [63]:
test_T_31['Y_Class'].values

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [73]:
df[df['Y_Class']==2]

,PRODUCT_ID,Y_Class
38,TEST_038,2
78,TEST_078,2
160,TEST_160,2
161,TEST_161,2
163,TEST_163,2
169,TEST_169,2
183,TEST_183,2
187,TEST_187,2
